# Vooray Application
Authors: Cody Crofoot, Jared Robinson, Henry Olsen, Grant Brinkerhoff

## Setting up the database

The script will create 4 different tables.
* item: sku specific information
* inv_level: inventory level information
* amz_price: selling price information for each order
* quantity_sold: sales information 


In [27]:
import mysql.connector
conn = mysql.connector.connect(host="localhost", user="root", database="vooray", password="PASSWORD")

curs = conn.cursor()
print(curs)


def create_item():
    curs.execute(
        f'''
        CREATE TABLE `vooray_c`.`item` (
          `fnsku` VARCHAR(60) NULL,
          `sku` VARCHAR(45) NOT NULL,
          `product_name` VARCHAR(360) NULL,
          `asin` VARCHAR(60),
          PRIMARY KEY (`sku`));
        ''')

def create_inv_level():
    curs.execute(
        f'''
        CREATE TABLE `vooray_c`.`inv_level` (
          `s_date` VARCHAR(60),
          `date` DATE NOT NULL,
          `quantity` INT NULL,
          `fulfillment_center_id` VARCHAR(20) NULL,
          `detailed_disposition` VARCHAR(45) NULL,
          `country` VARCHAR(10) NULL,
          `sku` VARCHAR(45) NULL);
        ''')

def create_amz_price():
    curs.execute(
        f'''
        CREATE TABLE `vooray_c`.`amz_price` (
          `sku` VARCHAR(45) NULL,
          `p_date` VARCHAR(60),
          `date` DATE NOT NULL,
          `amazon_order_id` VARCHAR(45) NULL,
          `fulfillment_channel` VARCHAR(45) NULL,
          `sales_channel` VARCHAR(45) NULL,
          `currency` VARCHAR(10) NULL,
          `item_price` FLOAT NULL,
          `item_tax` FLOAT NULL,
          `shipping_price` FLOAT NULL,
          `shipping_tax` FLOAT NULL,
          INDEX `sku_idx` (`sku` ASC) VISIBLE,
          CONSTRAINT `sku`
            FOREIGN KEY (`sku`)
            REFERENCES `vooray_c`.`item` (`sku`)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION);
        ''')


def create_quantity_sold():
    curs.execute(
        f'''
        CREATE TABLE `vooray_c`.`quantity_sold` (
          `sku` VARCHAR(45) NULL,
          `p_date` VARCHAR(60),
          `date` DATE NOT NULL,
          `quantity` INT NULL,
          `amazon_order_id` VARCHAR(45) NULL,
          `fulfillment_channel` VARCHAR(45) NULL,
          `sales_channel` VARCHAR(45) NULL,
          INDEX `sku_idx` (`sku` ASC) VISIBLE,
          CONSTRAINT ``
            FOREIGN KEY (`sku`)
            REFERENCES `vooray_c`.`item` (`sku`)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION);
        ''')
    
def drop_db():
    curs.execute(
        f'''
        DROP TABLE `vooray_c`.`amz_price`, `vooray_c`.`inv_level`, `vooray_c`.`item`, `vooray_c`.`quantity_sold`;
        ''')
    print("DATABASE DROPPED")


CMySQLCursor: (Nothing executed yet)


In [28]:
# drop_db()  # Drops the database, only use if resetting

create_item()
create_inv_level()
create_amz_price()
create_quantity_sold()

print("DATABASE CREATED")

DATABASE DROPPED
DATABASE CREATED


## Loading the database

The script will load the data from the two csvs into four Pandas Dataframes in order to upload it to the database. The script will clean the database and also check for duplicates when uploading to the database.

In [4]:
import pandas as pd
import numpy as np
import mysql.connector
import time
import sys

In [29]:
#############
# VARIABLES #
#############


# Use these variables if you want to run from the command line
# script = sys.argv[0]
# inv = sys.argv[1]
# prc = sys.argv[2]
# full_run = sys.argv[3]

# Use these if you want to run from the script
inv = r'inventory_demo.csv'
prc = r'price_demo.csv'
full_run = True

if full_run:
    num = 500000  # The amount of records you want to run through, for full run number is arbitrarily high
else:
    num = 10

#################


# END VARIABLES #
#################

In [30]:
print("STARTING")
start = time.time()

# Connect to the database
conn = mysql.connector.connect(host="localhost",
                               user="root",
                               database="vooray",
                               password="mis5900vooray")

curs = conn.cursor()
print(curs)

STARTING
CMySQLCursor: (Nothing executed yet)


In [38]:
####################################################
# CREATE TEMPLATE OF DATA FRAMES TO GO IN DATABASE #
####################################################

df_index = [0]

# item table
item_template = {'fnsku': np.nan,
                      'sku': np.nan,
                      'product_name': np.nan,
                      'asin': np.nan}
item = pd.DataFrame(item_template, df_index)

# inv_level table
inv_level_template = {'s_date': np.nan,
                      'date': np.nan,
                      'quantity': np.nan,
                      'fulfillment_center_id': np.nan,
                      'detailed_disposition': np.nan,
                      'country': np.nan,
                      'sku': np.nan}
inv_level = pd.DataFrame(inv_level_template, df_index)

# amz_price
amz_price_template = {'sku': np.nan,
                      'p_date': np.nan,
                      'date': np.nan,
                      'amazon_order_id': np.nan,
                      'fulfillment_channel': np.nan,
                      'sales_channel': np.nan,
                      'currency': np.nan,
                      'item_price': np.nan,
                      'item_tax': np.nan,
                      'shipping_price': np.nan,
                      'shipping_tax': np.nan}
amz_price = pd.DataFrame(amz_price_template, df_index)

# quantity sold
quantity_sold_template = {'sku': np.nan,
                          'date': np.nan,
                          'quantity': np.nan,
                          'amazon_order_id': np.nan,
                          'fulfillment_channel': np.nan,
                          'sales_channel': np.nan}
quantity_sold = pd.DataFrame(quantity_sold_template, df_index)


In [42]:
item.head()

,fnsku,sku,product_name,asin
0,NaN,NaN,NaN,NaN


In [43]:
inv_level.head()

,s_date,date,quantity,fulfillment_center_id,detailed_disposition,country,sku
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
amz_price.head()

,sku,p_date,date,amazon_order_id,fulfillment_channel,sales_channel,currency,item_price,item_tax,shipping_price,shipping_tax
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
quantity_sold.head()

,sku,date,quantity,amazon_order_id,fulfillment_channel,sales_channel
0,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
##########################################
# NEXT ACQUIRE THE DATA AND TRANSFORM IT #
##########################################

# get the data
inv_df = pd.read_csv(inv)
prc_df = pd.read_csv(prc)

# Remove white space
inv_df.columns = inv_df.columns.str.replace(' ', '')
prc_df.columns = prc_df.columns.str.replace(' ', '')


# print(list(prc_df))
prc_droplist = ['merchant-order-id', 'last-updated-date', 'order-status',
                'order-channel', 'url', 'ship-service-level', 'item-status', 'gift-wrap-price', 'gift-wrap-tax',
                'item-promotion-discount', 'ship-promotion-discount', 'ship-city', 'ship-state', 'ship-postal-code',
                'ship-country', 'promotion-ids']
prc_df = prc_df.drop(prc_droplist, axis=1)


# Fix date
inv_df['date'] = inv_df['snapshot-date'].str.split('T').str[0]
inv_df['date'] = inv_df['snapshot-date'].astype('datetime64[ns]')
prc_df['date'] = prc_df['purchase-date'].str.split('T').str[0]
prc_df['date'] = prc_df['purchase-date'].astype('datetime64[ns]')


In [63]:
#########################################
# STARTING WITH UPDATING THE ITEM TABLE #
#########################################
print("----STARTING ITEM-----")


# This info is found in both inventory count and amazon
# print(len(prc_df['sku'].unique()))
# print(len(inv_df['sku'].unique()))
sku_list = []

# Look over skus on inventory df
skus = inv_df['sku'].unique()
i = 1
for sku in skus:
    # Subset Data by sku
    df = inv_df[inv_df.sku == sku]
    df2 = prc_df[prc_df.sku == sku]

    # SET THE VARIABLES TO GO INTO THE DATABASE
    fnsku = df['fnsku'].unique()[0] if len(df['fnsku'].unique()) == 1 else ""
    sku = sku
    product_name = df['product-name'].unique()[0].replace('"','') if len(df['product-name'].unique()) == 1 else ""
    asin = df2['asin'].unique()[0] if len(df2['asin'].unique()) == 1 else ""

    temp_dict = {'fnsku': fnsku,
                 'sku': sku,
                 'product_name': product_name,
                 'asin': asin}
    temp_index = [i]
    i += 1

    temp = pd.DataFrame(temp_dict, temp_index)
    frames = [item, temp]
    item = pd.concat(frames)
    sku_list.append(sku)

# Now go add any leftovers from the prc_df
skus = prc_df['sku'].unique()
for sku in skus:
    if not (sku in sku_list):
        # Subset Data by sku
        df = inv_df[inv_df.sku == sku]
        df2 = prc_df[prc_df.sku == sku]
        fnsku = df['fnsku'].unique()[0] if len(df['fnsku'].unique()) == 1 else ""
        sku = sku
        product_name = df['product-name'].unique()[0].replace('"','') if len(df['product-name'].unique()) == 1 else ""
        asin = df2['asin'].unique()[0] if len(df2['asin'].unique()) == 1 else ""

        temp_dict = {'fnsku': fnsku,
                     'sku': sku,
                     'product_name': product_name,
                     'asin': asin}
        temp_index = [i]
        i += 1

        temp = pd.DataFrame(temp_dict, temp_index)
        frames = [item, temp]
        item = pd.concat(frames)

# Update the Database #
item.drop(item.index[:1], inplace=True)

for index, row in item.iterrows():

    # Test to see if this piece of data is already in the database
    curs.execute(
        f'''
        SELECT sku
        FROM `vooray_c`.`item`
        WHERE sku = "{row['sku']}"
        '''
    )
    results = curs.fetchall()

    # None of these should return more than one thing but just in case
    if (len(results) > 1):
        print(f"ERROR AT {row['sku']}")

    # If len is 0 that means it is not in the database
    if len(results) == 0:
        curs.execute(
            f'''
            INSERT INTO `vooray_c`.`item`(fnsku, sku, product_name, asin)
            VALUES ("{row['fnsku']}", "{row['sku']}", "{row['product_name']}", "{row['asin']}");
            '''
        )
        conn.commit()

print("ITEM TABLE UPLOADED")

----STARTING ITEM-----
ITEM TABLE UPLOADED


In [64]:
item.head()

,fnsku,sku,product_name,asin
3,fnsku3,sku3,product3,asin3
4,fnsku4,sku4,product4,
5,fnsku5,sku5,product5,asin5
6,fnsku6,sku6,product6,asin6
7,fnsku7,sku7,product7,asin7


In [65]:
################################
# Updating the Inv_Level Table #
################################
print("----STARTING INV_LEVEL-----")


snapshots = inv_df['snapshot-date'].unique()
skus = inv_df['sku'].unique()
i = 1
for snapshot in snapshots:
    filter1 = inv_df['snapshot-date'] == snapshot
    df = inv_df[filter1]

    for index, row in df.iterrows():
        s_date = row['snapshot-date']
        date = row['date']
        quantity = row['quantity']
        fulfillment_center_id = row['fulfillment-center-id']
        detailed_disposition = row['detailed-disposition']
        country = row['country']
        sku = row['sku']

        temp_dict = {'s_date': s_date,
                     'date': date,
                     'quantity': quantity,
                     'fulfillment_center_id': fulfillment_center_id,
                     'detailed_disposition': detailed_disposition,
                     'country': country,
                     'sku': sku}
        temp_index = [i]
        i += 1
        if i % 1000 == 0:
            print("INV: ", i)

        temp = pd.DataFrame(temp_dict, temp_index)
        frames = [inv_level, temp]
        inv_level = pd.concat(frames)
        if i == num:
            # print(inv_level)
            break
    if i == num:
        break

#update the database
inv_level.drop(inv_level.index[:1], inplace=True)

# print(inv_level.head())
# print(list(inv_level))
for index, row in inv_level.iterrows():
    # Test to see if this piece of data is already in the database
    curs.execute(
        f'''
        SELECT sku, date, fulfillment_center_id
        FROM `vooray_c`.`inv_level`
        WHERE sku = "{row['sku']}" AND fulfillment_center_id = "{row['fulfillment_center_id']}" AND DATE(date) = DATE("{row['date']}") 
        '''
    )
    results = curs.fetchall()
    # None of these should return more than one thing but just in case
    if (len(results) > 1):
        print(f"DUPLICATE AT {row}")

    # If len is 0 that means it is not in the database
    if len(results) == 0:
        curs.execute(
            f'''
            INSERT INTO `vooray_c`.`inv_level`( s_date, date, quantity, fulfillment_center_id, detailed_disposition, country, sku)
            VALUES ( "{row['s_date']}", "{row['date']}", "{row['quantity']}", "{row['fulfillment_center_id']}", "{row['detailed_disposition']}", "{row['country']}", "{row['sku']}");
            '''
        )
        conn.commit()
        
        
print("INV_LEVEL TABLE UPLOADED")

----STARTING INV_LEVEL-----
INV_LEVEL TABLE UPLOADED


In [66]:
inv_level.head()

,s_date,date,quantity,fulfillment_center_id,detailed_disposition,country,sku
4,2018-10-31T07:00:00+00:03,2018-10-31 06:57:00,77.0,loc4,SELLABLE,US,sku4
5,2018-10-31T07:00:00+00:04,2018-10-31 06:56:00,68.0,loc5,SELLABLE,US,sku5
6,2018-10-31T07:00:00+00:05,2018-10-31 06:55:00,11.0,loc6,SELLABLE,UK,sku6
7,2018-10-31T07:00:00+00:06,2018-10-31 06:54:00,37.0,loc7,SELLABLE,US,sku7
8,2018-10-31T07:00:00+00:07,2018-10-31 06:53:00,46.0,loc8,DEFECTIVE,US,sku8


In [67]:
##########################
# Updating the AMZ_Price #
##########################
print("----STARTING AMZ_PRICE-----")
dates = prc_df['purchase-date'].unique()
i = 1

for date in dates:
    filter1 = prc_df['purchase-date'] == date
    df = prc_df[filter1]
    for index, row in df.iterrows():
        sku = row['sku']
        p_date = row['purchase-date']
        date = row['date']
        amazon_order_id = row['amazon-order-id']
        fulfillment_channel = row['fulfillment-channel']
        sales_channel = row['sales-channel']
        currency = row['currency']
        item_price = row['item-price'] if not np.isnan(row['item-price']) else 0
        item_tax = row['item-tax']if not np.isnan(row['item-tax']) else 0
        shipping_price = row['shipping-price'] if not np.isnan(row['shipping-price']) else 0
        shipping_tax = row['shipping-tax'] if not np.isnan(row['shipping-tax']) else 0

        temp_dict = {'sku': sku,
                      'p_date': p_date,
                      'date': date,
                      'amazon_order_id': amazon_order_id,
                      'fulfillment_channel': fulfillment_channel,
                      'sales_channel': sales_channel,
                      'currency': currency,
                      'item_price': item_price,
                      'item_tax': item_tax,
                      'shipping_price': shipping_price,
                      'shipping_tax': shipping_tax}
        temp_index = [i]
        i += 1
        if i % 1000 == 0:
            print("PRICE: ", i)


        temp = pd.DataFrame(temp_dict, temp_index)
        frames = [amz_price, temp]
        amz_price = pd.concat(frames)
        if i == num:
            break
    if i == num:
        break

# Update the Database #
amz_price.drop(amz_price.index[:1], inplace=True)

# print(amz_price.head())
# print(list(amz_price))
# print(amz_price)
# print(amz_price['amazon_order_id'])

for index, row in amz_price.iterrows():

    # Test to see if this piece of data is already in the database
    curs.execute(
        f'''
        SELECT sku
        FROM `vooray_c`.`amz_price`
        WHERE sku = "{row['sku']}" AND amazon_order_id = "{row['amazon_order_id']}" AND DATE(date) = DATE("{row['date']}") 
        '''
    )
    results = curs.fetchall()

    # None of these should return more than one thing but just in case
    if (len(results) > 1):
        print(f"DUPLICATE AT {row['sku']}")

    # If len is 0 that means it is not in the database
    if len(results) == 0:
        curs.execute(
            f'''
            INSERT INTO `vooray_c`.`amz_price`(sku, p_date, date, amazon_order_id, fulfillment_channel, sales_channel, currency, item_price, item_tax, shipping_price, shipping_tax)
            VALUES ("{row['sku']}", "{row['p_date']}", "{row['date']}", "{row['amazon_order_id']}", "{row['fulfillment_channel']}", "{row['sales_channel']}", "{row['currency']}", "{row['item_price']}", "{row['item_tax']}", "{row['shipping_price']}", "{row['shipping_tax']}");
            '''
        )
        conn.commit()

print("AMZ_PRICE TABLE UPLOADED")

----STARTING AMZ_PRICE-----
AMZ_PRICE TABLE UPLOADED


In [68]:
amz_price.head()

,sku,p_date,date,amazon_order_id,fulfillment_channel,sales_channel,currency,item_price,item_tax,shipping_price,shipping_tax
3,sku3,2018-10-15T08:35:15+00:02,2018-10-15 08:33:15,000-0000000-0000003,Amazon,Amazon.com,USD,18.99,2.0,1.0,0.0
4,sku4,2018-10-15T08:35:15+00:03,2018-10-15 08:32:15,000-0000000-0000004,Amazon,Amazon.com,USD,20.99,2.0,3.0,0.0
5,sku5,2018-10-15T08:35:15+00:04,2018-10-15 08:31:15,000-0000000-0000005,Amazon,Amazon.ca,CAD,22.99,3.0,0.0,0.0
6,sku6,2018-10-15T08:35:15+00:05,2018-10-15 08:30:15,000-0000000-0000006,Amazon,Amazon.com,USD,24.99,4.0,0.0,0.0
7,sku7,2018-10-15T08:35:15+00:06,2018-10-15 08:29:15,000-0000000-0000007,Amazon,Amazon.com,USD,26.99,3.0,2.0,0.0


In [69]:
####################################
# Updating the quantity_sold Table #
####################################
print("----STARTING QUANTITY_SOLD-----")


snapshots = prc_df['purchase-date'].unique()
skus = prc_df['sku'].unique()
# print(len(snapshots))
i = 1
for snapshot in snapshots:
    filter1 = prc_df['purchase-date'] == snapshot
    df = prc_df[filter1]

    for index, row in df.iterrows():
        sku = row['sku']
        p_date = row['purchase-date']
        date = row['date']
        quantity = row['quantity']
        amazon_order_id = row['amazon-order-id']
        fulfillment_channel = row['fulfillment-channel']
        sales_channel = row['sales-channel']

        temp_dict = {'sku': sku,
                     'p_date': p_date,
                     'date': date,
                     'quantity': quantity,
                     'amazon_order_id': amazon_order_id,
                     'fulfillment_channel': fulfillment_channel,
                     'sales_channel': sales_channel}
        temp_index = [i]
        i += 1
        if i % 1000 == 0:
            print("QUANTITY: ", i)

        temp = pd.DataFrame(temp_dict, temp_index)
        frames = [quantity_sold, temp]
        quantity_sold = pd.concat(frames, sort=True)
        if i == num:
            # print(quantity_sold)
            break
    if i == num:
        break

    # update the database
quantity_sold.drop(quantity_sold.index[:1], inplace=True)

# print(quantity_sold.he
for index, row in quantity_sold.iterrows():
    # Test to see if this piece of data is already in the database
    curs.execute(
        f'''
        SELECT sku, date, quantity, amazon_order_id
        FROM `vooray_c`.`quantity_sold`
        WHERE sku = "{row['sku']}" AND quantity = "{row['quantity']}" AND DATE(date) = DATE("{row['date']}") AND amazon_order_id = "{row['amazon_order_id']}"
        '''
    )
    results = curs.fetchall()
    # None of these should return more than one thing but just in case
    if (len(results) > 1):
        print(f"DUPLICATE AT {row}")

    # If len is 0 that means it is not in the database
    if len(results) == 0:
        curs.execute(
            f'''
            INSERT INTO `vooray_c`.`quantity_sold`( sku, p_date, date, quantity, amazon_order_id, fulfillment_channel, sales_channel)
            VALUES ( "{row['sku']}", "{row['p_date']}", "{row['date']}", "{row['quantity']}", "{row['amazon_order_id']}", "{row['fulfillment_channel']}", "{row['sales_channel']}");
            '''
        )
        conn.commit()
        
        
print("QUANTITY_SOLD TABLE UPLOADED")

----STARTING QUANTITY_SOLD-----
QUANTITY_SOLD TABLE UPLOADED


In [70]:
quantity_sold.head()

,amazon_order_id,date,fulfillment_channel,p_date,quantity,sales_channel,sku
3,000-0000000-0000003,2018-10-15 08:33:15,Amazon,2018-10-15T08:35:15+00:02,1.0,Amazon.com,sku3
4,000-0000000-0000004,2018-10-15 08:32:15,Amazon,2018-10-15T08:35:15+00:03,1.0,Amazon.com,sku4
5,000-0000000-0000005,2018-10-15 08:31:15,Amazon,2018-10-15T08:35:15+00:04,1.0,Amazon.ca,sku5
6,000-0000000-0000006,2018-10-15 08:30:15,Amazon,2018-10-15T08:35:15+00:05,1.0,Amazon.com,sku6
7,000-0000000-0000007,2018-10-15 08:29:15,Amazon,2018-10-15T08:35:15+00:06,1.0,Amazon.com,sku7


In [37]:
end = time.time()

print("SUCCESS!")
print(end - start)

SUCCESS!
18.859985828399658
